In [32]:
import numpy as np 
import pandas as pd
from tqdm import tqdm
import os
import json
from bigtree import dict_to_tree
from bigtree import tree_to_dot
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


In [2]:
path = 'C:/Users/18565/Desktop/Classes/VA/Project/Code/VA-Project'
file_ = open(os.path.join(path,'data/json_data.json'))
data_dic = json.loads(file_.read())

In [36]:
def create_tree(data,ignore_car_id=None,get_analomy=None):
    tree = {"start":{"value":4,'time':0}}
    for car in data:
        if ignore_car_id:
            if car['car-id'] in ignore_car_id:
                # print("here")
                continue
        if get_analomy and ignore_car_id==None:
            if car['car-id'] not in get_analomy:
                continue
        route = car['path_taken'][0]
        key = "start/"+list(route.keys())[0].split(" ")[0]
        if key in tree.keys():
            tree[key]['value']+=1
        else:
            tree[key]={"value":1,'time':0}
        
        for route in car['path_taken']:
            temp =list(route.keys())[0].split(" ")[-1]
            key+="/"
            key+= temp
            if key in tree.keys():
                tree[key]['value']+=1
                tree[key]['time']+=list(route.values())[0]
            else:
                tree[key]={"value":1,'time':list(route.values())[0]}
    return tree

# Routes Analysis of car type 1

In [160]:
root_dic = create_tree(data_dic['data']['1'])

In [102]:
root = dict_to_tree(root_dic)

In [104]:
graph = tree_to_dot(root)

In [105]:
graph.write_png(os.path.join(path,"data/fig/cartype1.png"))

We noticed 3 long routes in the Tree figure of Car type 1. Analyzing more on that

In [39]:
def get_max_length_routes(data,num_routes):
    max_lengths = [0]*num_routes
    car_ids = [0]*num_routes
    for car in data:
        for i in range(num_routes):
            if max_lengths[i] < len(car['path_taken']):
                max_lengths[i] = len(car['path_taken'])
                car_ids[i] = car['car-id']
                break
    return car_ids,max_lengths

In [125]:
max_length_route_cars_ids, max_length = get_max_length_routes(data_dic['data']["1"],3)

In [127]:
max_length_route_cars_ids,max_length

(['20154112014114-381', '20155705025759-63', '20162904122951-717'],
 [97, 69, 35])

In [45]:
df = pd.read_csv(os.path.join(path,"data/Lekagul Sensor Data.csv"))

In [46]:
def get_analomy_df(df,max_length_route_cars_ids,cartype):
    for car_id in max_length_route_cars_ids:
        car_df = df[df['car-id']==car_id]
        in_time = car_df['Timestamp'].iloc[0]
        out_time = car_df['Timestamp'].iloc[-1]
        car_df.to_csv(os.path.join(path,"data/anolomy/stayed_to_long/catype"+cartype+"_entries_"+str(len(car_df))+"_"+car_id+".csv"))
        print((car_id,in_time,out_time,len(car_df)))
        
   

In [ ]:
get_analomy_df(df,max_length_route_cars_ids)

In [145]:
tree_without_analomy = create_tree(data_dic['data']['1'],max_length_route_cars_ids)

In [141]:
root = dict_to_tree(tree_without_analomy)
graph = tree_to_dot(root)

In [142]:
graph.write_png(os.path.join(path,"data/fig/cartype1_without_analomy.png"))

In [146]:
tree_analomy = create_tree(data_dic['data']['1'],None,max_length_route_cars_ids)

In [149]:
root = dict_to_tree(tree_analomy)
graph = tree_to_dot(root)

In [151]:
graph.write_png(os.path.join(path,"data/fig/cartype1_analomy.png"))

# Route Analysis of car type 2p

In [167]:
root_dic = create_tree(data_dic['data']['2P'])
root = dict_to_tree(root_dic)
graph = tree_to_dot(root)
graph.write_png(os.path.join(path,"data/fig/cartype2P.png"))

# Route Analysis of Car type 2

In [38]:
root_dic = create_tree(data_dic['data']['2'])
root = dict_to_tree(root_dic)
graph = tree_to_dot(root)
graph.write_png(os.path.join(path,"data/fig/cartype2.png"))

In [57]:
max_length_route_cars_ids, max_length = get_max_length_routes(data_dic['data']["2"],1)

In [58]:
get_analomy_df(df,max_length_route_cars_ids,"2")

('20154519024544-322', '2015-06-19 14:45:44', '2015-10-05 00:36:54', 281)


In [60]:
tree_without_analomy = create_tree(data_dic['data']['2'],max_length_route_cars_ids)
root = dict_to_tree(tree_without_analomy)
graph = tree_to_dot(root)
graph.write_png(os.path.join(path,"data/fig/cartype2_without_analomy.png"))

In [62]:
analomy = create_tree(data_dic['data']['2'],None,max_length_route_cars_ids)
root = dict_to_tree(analomy)
graph = tree_to_dot(root)
graph.write_png(os.path.join(path,"data/fig/cartype2_analomy.png"))

# Route Analysis of Car Type 3

In [63]:
root_dic = create_tree(data_dic['data']['3'])
root = dict_to_tree(root_dic)
graph = tree_to_dot(root)
graph.write_png(os.path.join(path,"data/fig/cartype3.png"))

# Route Analysis of Car Type 4

In [68]:
root_dic = create_tree(data_dic['data']['4'])
root = dict_to_tree(root_dic)
graph = tree_to_dot(root)
graph.write_png(os.path.join(path,"data/fig/cartype4.png"))

# Route Analysis of Car Type 5

In [95]:
root_dic = create_tree(data_dic['data']['5'])
root = dict_to_tree(root_dic)
graph = tree_to_dot(root)
graph.write_png(os.path.join(path,"data/fig/cartype5.png"))

# Route Analysis of Car Type 6

In [96]:
root_dic = create_tree(data_dic['data']['6'])
root = dict_to_tree(root_dic)
graph = tree_to_dot(root)
graph.write_png(os.path.join(path,"data/fig/cartype6.png"))

* cartype 5 and 6 take the same routes 